<img src="https://res.cloudinary.com/dbl53sidm/image/upload/v1696398508/mistral-7b-v0.1_opibjl.jpg" width="100%">

## Instruct Fine-tuning [Mistral 7B Instruct](https://mistral.ai/news/announcing-mistral-7b/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the Mistral-7b-Instruct Model

## Meet Mistral 7B Instruct

The team at [MistralAI](https://mistral.ai/news/announcing-mistral-) has created an exceptional language model called Mistral 7B Instruct. It has consistently delivered outstanding results in a range of benchmarks, which positions it as an ideal option for natural language generation and understanding. This guide will concentrate on how to fine-tune the model for coding purposes, but the methodology can effectively be applied to other tasks.

All the code will be available on the Github [adithya-s-k](https://github.com/adithya-s-k)

## Prerequisites

Before diving into the fine-tuning process, make sure you have the following prerequisites in place:

1. **GPU**: While this tutorial can run on a free Google Colab notebook with a GPU, it's recommended to use more powerful GPUs like V100 or A100 for better performance.
2. **Python Packages**: Ensure you have the required Python packages installed. You can run the following commands to install them:

Let's start by checking if your GPU is correctly detected:

In [1]:
!nvidia-smi

Wed Nov 22 05:46:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Lets Get Started

Let install the python packages

In [1]:
!pip install -q torch
!pip install -q git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install -q peft #Parameter efficient finetuning - for qLora Finetuning
!pip install -q bitsandbytes #For Model weights quantisation
!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
!pip install -q wandb -U #Used to monitor the model score during training
!pip install -q -U gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset, load_from_disk
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer # For supervised finetuning

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

### Lets Load the Dataset

We will fine-tune Mistral 7B Instruct for Conversational Chatbot.

We will be using this [dataset](https://drive.google.com/file/d/1-0EVUR6gD-lbhIO29-6e_8PHtE7aOYnZ/view?usp=share_link). The dataset structure should resemble the following:

```json
{
  "input": "your prompt/query/question",
  "response": "your lable",
  "text": "combination of `input` and `response`"
}
```
user's utterance is wrapped by `[INST]` and `[/INST]` special token. On the other hand, user's utterance is seemed to be an instruction for the bot

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
# https://drive.google.com/file/d/1Uv1f_jRYL53afitgLA8KTP2LCDBCTMNz/view?usp=drive_link
!gdown 1Uv1f_jRYL53afitgLA8KTP2LCDBCTMNz

Downloading...
From: https://drive.google.com/uc?id=1Uv1f_jRYL53afitgLA8KTP2LCDBCTMNz
To: /content/question_dataset.zip
100% 5.06M/5.06M [00:00<00:00, 39.0MB/s]


In [5]:
!unzip /content/question_dataset.zip

Archive:  /content/question_dataset.zip
   creating: question_dataset/
 extracting: question_dataset/dataset_dict.json  
   creating: question_dataset/train/
  inflating: question_dataset/train/data-00000-of-00001.arrow  
  inflating: question_dataset/train/dataset_info.json  
  inflating: question_dataset/train/state.json  
   creating: question_dataset/validation/
  inflating: question_dataset/validation/data-00000-of-00001.arrow  
  inflating: question_dataset/validation/dataset_info.json  
  inflating: question_dataset/validation/state.json  


In [6]:
dataset = load_from_disk("/content/question_dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['field', 'entities', 'complex_question', 'triplets', 'answer'],
        num_rows: 28856
    })
    validation: Dataset({
        features: ['field', 'entities', 'complex_question', 'triplets', 'answer'],
        num_rows: 7214
    })
})

In [9]:
dataset["train"][0]["entities"]

[]

In [10]:
dataset["train"][0]["answer"]

''

In [11]:
dataset["train"][0]["complex_question"]

'Bạn ơi sắp tới công ty mình hợp tác với nhà đầu tư nước ngoài, không có trụ sở hay chi nhánh ở Việt Nam. Bên mình làm dịch vụ lắp ráp máy móc cho công ty đó địa điểm lắp máy tại Việt Nam. Ban tư vấn cho mình hỏi khi hợp tác với nhà đầu tư nước ngoài như vậy thì cần lưu ý những vấn đề gì?'

In [12]:
dataset["train"][0]["triplets"]

[{'question': 'Khi doanh nghiệp trong nước hợp tác với nhà đầu tư nước ngoài thì cần phải lưu ý những quy định nào?'},
 {'question': 'Nhà đầu tư nước ngoài được áp dụng điều kiện tiếp cận thị trường như quy định đối với nhà đầu tư trong nước không?'},
 {'question': 'Khi nhà đầu tư nước ngoài và nhà đầu tư trong nước có tranh chấp thì việc giải quyết được thực hiện ở đâu?'}]

In [7]:
train_dataset = dataset["train"]

In [8]:
# prompt_template = """Decompose the complex question to multiple simple questions.
# Each generated simple question is represented each sub-problem from the complex question so that after answering we have the necessary knowledge to answer the complex question.
# Complex question:
# {complex_question}
# Simple questions:"""
prompt_template = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật.
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản.
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: {complex_question}
### Assistant:"'''

In [15]:
triples_sample = train_dataset["triplets"]
triples_sample

[[{'question': 'Khi doanh nghiệp trong nước hợp tác với nhà đầu tư nước ngoài thì cần phải lưu ý những quy định nào?'},
  {'question': 'Nhà đầu tư nước ngoài được áp dụng điều kiện tiếp cận thị trường như quy định đối với nhà đầu tư trong nước không?'},
  {'question': 'Khi nhà đầu tư nước ngoài và nhà đầu tư trong nước có tranh chấp thì việc giải quyết được thực hiện ở đâu?'}],
 [{'question': 'Phẫu thuật làm mỏm cụt ngón được hiểu như thế nào?'},
  {'question': 'Khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?'},
  {'question': 'Phẫu thuật làm mỏm cụt ngón thì quy trình tiến hành như thế nào?'}],
 [{'question': 'Lễ Phật Đản 2023 vào ngày nào?'},
  {'question': 'Nguyên tắc tổ chức Lễ Phật Đản như thế nào?'},
  {'question': 'Ban tổ chức Lễ Phật Đản phải có trách nhiệm như thế nào?'},
  {'question': 'Tổ chức lễ Phật Đản có phải thông báo đến cơ quan có thẩm quyền không?'}],
 [{'question': 'Hướng dẫn kiểm tra điều kiện kiểm tra, giám sát hải quan đối với doanh nghiệp chế xuất theo Nghị

In [16]:
simple_question_samples = [[t["question"] for t in triple] for triple in triples_sample]

In [17]:
simple_question_samples

[['Khi doanh nghiệp trong nước hợp tác với nhà đầu tư nước ngoài thì cần phải lưu ý những quy định nào?',
  'Nhà đầu tư nước ngoài được áp dụng điều kiện tiếp cận thị trường như quy định đối với nhà đầu tư trong nước không?',
  'Khi nhà đầu tư nước ngoài và nhà đầu tư trong nước có tranh chấp thì việc giải quyết được thực hiện ở đâu?'],
 ['Phẫu thuật làm mỏm cụt ngón được hiểu như thế nào?',
  'Khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?',
  'Phẫu thuật làm mỏm cụt ngón thì quy trình tiến hành như thế nào?'],
 ['Lễ Phật Đản 2023 vào ngày nào?',
  'Nguyên tắc tổ chức Lễ Phật Đản như thế nào?',
  'Ban tổ chức Lễ Phật Đản phải có trách nhiệm như thế nào?',
  'Tổ chức lễ Phật Đản có phải thông báo đến cơ quan có thẩm quyền không?'],
 ['Hướng dẫn kiểm tra điều kiện kiểm tra, giám sát hải quan đối với doanh nghiệp chế xuất theo Nghị định 35/2022/NĐ-CP?',
  'Hướng dẫn kiểm tra hoạt động thuê kho bên ngoài khu công nghiệp, khu kinh tế của doanh nghiệp chế xuất?',
  'Hướng dẫn xác định

In [9]:
def transform(examples):
    simple_question_lst = [[t["question"] for t in triple] for triple in examples["triplets"]]
    # text = ["<s> [INST] " + prompt_template.format(complex_question=cq)+ "[/INST]\n" + "\n".join(sq) + " </s>"
    #         for cq, sq in zip(examples["complex_question"], simple_question_lst)]
    text = ["<s> " + prompt_template.format(complex_question=cq)+ "\n" + "\n".join(sq) + '" </s>'
            for cq, sq in zip(examples["complex_question"], simple_question_lst)]
    examples["text"] = text
    return examples

In [10]:
train_dataset = train_dataset.map(transform, batched=True)

Map:   0%|          | 0/28856 [00:00<?, ? examples/s]

In [11]:
val_dataset = dataset['validation']
val_dataset = val_dataset.map(transform, batched=True)

Map:   0%|          | 0/7214 [00:00<?, ? examples/s]

In [12]:
print(train_dataset["text"][1])

<s> Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi rằng phẫu thuật làm mỏm cụt ngón được hiểu như thế nào? Bên cạnh đó thì cho hỏi thêm là khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?
### Assistant:"
Phẫu thuật làm mỏm cụt ngón được hiểu như thế nào?
Khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?
Phẫu thuật làm mỏm cụt ngón thì quy trình tiến hành như thế nào?" </s>


In [13]:
print(val_dataset["text"][1])

<s> Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Doanh nghiệp anh có một nhà xưởng khung thép tiền chế, anh muốn biết có cần nộp lệ phí trước bạ đối với nhà xưởng này hay không? Bên cạnh đó, doanh nghiệp anh có mảnh đất thì không biết có phải nộp lệ phí trước bạ đối với mảnh đất này không?
### Assistant:"
Có thể được miễn lệ phí trước bạ đối với nhà xưởng khung thép tiền chế của doanh nghiệp hay không?
Doanh nghiệp có phải nộp lệ phí trước bạ đối với phần đất được giao sở hữu hay không?
Giá tính lệ phí trước bạ đối với đất được xây dựng dựa trên cơ sở nào?" </s>


## Setting Model Parameters

We need to set various parameters for our fine-tuning process, including QLoRA (Quantization LoRA) parameters, bitsandbytes parameters, and training arguments:

In [14]:
# The model that you want to train from the Hugging Face hub
model_name = "vilm/vietcuna-7b-v3" #"mistralai/Mistral-7B-Instruct-v0.1"

# Fine-tuned model name
new_model = "Pythera-question-decomposition" #"mistralai-Code-Instruct"

Setting the QLora Parameters

In [26]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 128

# Alpha parameter for LoRA scaling
lora_alpha = 32

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 100

do_eval = False
evaluation_strategy = 'steps'

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

### Lets Load the base model
Let's load the Mistral 7B Instruct base model:

In [16]:
# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MistralAI tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

In [17]:
base_model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear4bit(in_features=4096, out_features=12288, bias=True)
          (dense): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear4bit(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear4bit(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_a

## Let's Check how the base model performs


In [27]:
train_dataset[5]["complex_question"]

'Cho tôi hỏi: Mẫu quyết định cưỡng chế trả vật trong thi hành án dân sự mới nhất theo Thông tư 04/2023 / TT-BTP là mẫu nào?'

In [28]:
eval_simple_questions = [t["question"] for t in train_dataset[5]["triplets"]]
eval_simple_questions

['Mẫu Quyết định cưỡng chế trả vật trong thi hành án dân sự mới nhất theo Thông tư 04/2023/TT-BTP?',
 'Thủ tục cưỡng chế trả vật trong thi hành án dân sự được quy định ra sao?',
 'Thông tư 04/2023/TT-BTP được áp dụng từ ngày mấy?']

In [29]:
tokenizer.pad_token_id

2

## Zero shot

In [30]:
### VietCuna
def first_evaluate(complex_question):
    # eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng. Người dùng nhập câu hỏi phức tạp, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản sinh ra được thể hiện từng vấn đề con của câu hỏi phức tạp để sau khi trả lời chúng ta có đủ kiến thức cần thiết để trả lời câu hỏi phức tạp.\n\n### Human: {human_message}\n\n### Assistant:"'''
    # eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.\n\n### Human: {human_message}\n\n### Assistant:"'''
    # eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp.\n\n### Human: {human_message}\n\n### Assistant:"'''
    eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.\n\n### Human: {human_message}\n\n### Assistant:"'''

    input_text = eval_prompt.format(human_message=complex_question)
    model_input = tokenizer(input_text, return_tensors="pt").to("cuda")

    base_model.eval()
    with torch.no_grad():
        output = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True)
    return output

In [31]:
complex_question = train_dataset[0]["complex_question"]
print(first_evaluate(complex_question))
print(train_dataset[0]["triplets"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1554: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.

### Human: Bạn ơi sắp tới công ty mình hợp tác với nhà đầu tư nước ngoài, không có trụ sở hay chi nhánh ở Việt Nam. Bên mình làm dịch vụ lắp ráp máy móc cho công ty đó địa điểm lắp máy tại Việt Nam. Ban tư vấn cho mình hỏi khi hợp tác với nhà đầu tư nước ngoài như vậy thì cần lưu ý những vấn đề gì?

### Assistant:"Khi hợp tác với một nhà đầu tư nước ngoài không có trụ sở hoặc chi nhánh tại Việt Nam, bạn cần lưu ý những vấn đề sau:

1. Hợp đồng: Hợp đồng giữa bạn và nhà đầu tư nước ngoài nên được soạn thảo và ký kết cẩn thận. Nó nên bao gồm các điều khoản về trách nhiệm pháp lý, bảo mật thông tin và quyền sở hữu trí tuệ. Ng

In [32]:
complex_question = train_dataset[1]["complex_question"]
print(first_evaluate(complex_question))
print(train_dataset[1]["triplets"])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.

### Human: Cho hỏi rằng phẫu thuật làm mỏm cụt ngón được hiểu như thế nào? Bên cạnh đó thì cho hỏi thêm là khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?

### Assistant:"Phẫu thuật làm mỏm cụt ngón" được hiểu là một thủ thuật trong đó bác sĩ phẫu thuật loại bỏ một phần hoặc toàn bộ phần mỏm cụt của ngón. Phẫu thuật này thường được thực hiện để điều trị các vấn đề về ngón như ngón bị gãy hoặc ngón bị biến dạng. Phẫu thuật làm mỏm cụt ngón thường được thực hiện khi ngón bị gãy hoặc biến dạng, và nó có thể được thực hiện để phục hồi chức năng hoặc để bảo tồn ngón. Phẫu thuật làm mỏm cụt ngón thường được thực hiện trong 

In [33]:
complex_question = train_dataset[2]["complex_question"]
print(first_evaluate(complex_question))
print(train_dataset[2]["triplets"])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.

### Human: Tôi muốn hỏi Lễ Phật Đản 2023 vào ngày nào?

### Assistant:"Lễ Phật Đản 2023 rơi vào ngày nào?"
[{'question': 'Lễ Phật Đản 2023 vào ngày nào?'}, {'question': 'Nguyên tắc tổ chức Lễ Phật Đản như thế nào?'}, {'question': 'Ban tổ chức Lễ Phật Đản phải có trách nhiệm như thế nào?'}, {'question': 'Tổ chức lễ Phật Đản có phải thông báo đến cơ quan có thẩm quyền không?'}]


In [34]:
complex_question = train_dataset[-1]["complex_question"]
print(first_evaluate(complex_question))
print(train_dataset[-1]["triplets"])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.

### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?

### Assistant:" Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ gì?"
[{'question': 'Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?'}, {'question': 'Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?'}, {'question': 'Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?'}]


In [35]:
complex_question = train_dataset[2]["complex_question"]
print(first_evaluate(complex_question))
print(train_dataset[2]["triplets"])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.

### Human: Tôi muốn hỏi Lễ Phật Đản 2023 vào ngày nào?

### Assistant:"Lễ Phật Đản 2023 rơi vào ngày nào?"
[{'question': 'Lễ Phật Đản 2023 vào ngày nào?'}, {'question': 'Nguyên tắc tổ chức Lễ Phật Đản như thế nào?'}, {'question': 'Ban tổ chức Lễ Phật Đản phải có trách nhiệm như thế nào?'}, {'question': 'Tổ chức lễ Phật Đản có phải thông báo đến cơ quan có thẩm quyền không?'}]


## One shot

In [36]:
print(train_dataset[-1]["complex_question"])
[t["question"] for t in train_dataset[-1]["triplets"]]

Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?


['Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?',
 'Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?',
 'Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?']

In [37]:
def oneshot_evaluate(complex_question):
    # eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật.
    # Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản.
    # Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp.
    # ### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
    # ### Assistant:"
    # 1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
    # 2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
    # 3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

    # ### Human: {human_message}
    # ### Assistant:"'''

    eval_prompt = '''Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật.
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản.
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
### Assistant:"
1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

### Human: {human_message}
### Assistant:"'''
    input_text = eval_prompt.format(human_message=complex_question)
    model_input = tokenizer(input_text, return_tensors="pt").to("cuda")

    base_model.eval()
    with torch.no_grad():
        output = tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True)
    return output

In [38]:
complex_question = train_dataset[1]["complex_question"]
print(oneshot_evaluate(complex_question))
print([t["question"] for t in train_dataset[1]["triplets"]])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
### Assistant:"
1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

### Human: Cho hỏi rằng phẫu thuật làm mỏm cụt ngón được hiểu như thế nào? Bên cạnh đó thì cho hỏi thêm là khi nào người bệnh được phẫu thuật làm mỏm cụt ngón?
### Assistant:"
1. Phẫu thuật làm mỏm cụt ngón là gì?
2. Phẫu thuật làm mỏm cụt ngón được hiểu như thế

In [39]:
complex_question = train_dataset[0]["complex_question"]
print(oneshot_evaluate(complex_question))
print([t["question"] for t in train_dataset[0]["triplets"]])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
### Assistant:"
1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

### Human: Bạn ơi sắp tới công ty mình hợp tác với nhà đầu tư nước ngoài, không có trụ sở hay chi nhánh ở Việt Nam. Bên mình làm dịch vụ lắp ráp máy móc cho công ty đó địa điểm lắp máy tại Việt Nam. Ban tư vấn cho mình hỏi khi hợp tác với nhà đầu tư nước ngoài n

In [40]:
complex_question = train_dataset[2]["complex_question"]
print(oneshot_evaluate(complex_question))
print([t["question"] for t in train_dataset[2]["triplets"]])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
### Assistant:"
1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

### Human: Tôi muốn hỏi Lễ Phật Đản 2023 vào ngày nào?
### Assistant:"
1. Lễ Phật Đản 2023 rơi vào ngày nào?
2. Lễ Phật Đản 2023 có được tổ chức trong một cơ quan hành chính nhà nước không?"
['Lễ Phật Đản 2023 vào ngày nào?', 'Nguyên tắc tổ chức Lễ Phật Đản như 

In [41]:
complex_question = train_dataset[5]["complex_question"]
print(oneshot_evaluate(complex_question))
print([t["question"] for t in train_dataset[5]["triplets"]])

Bạn là trợ lý trí tuệ nhân tạo hỗ trợ người dùng về các vấn đề về luật. 
Người dùng nhập câu hỏi phức tạp có liên quan đến luật, bạn sẽ phân tách câu hỏi phức tạp thành nhiều câu hỏi đơn giản. 
Mỗi câu hỏi đơn giản được sinh ra thể hiện từng vấn đề con liên quan đến luật của câu hỏi phức tạp để có thể sử dụng các kiến thức về luật trả lời cho từng câu hỏi đơn giản đó.
### Human: Cho hỏi Kế toán viên trong cơ quan hành chính nhà nước phải có trình độ như thế nào?
### Assistant:"
1. Kế toán viên trong cơ quan hành chính nhà nước có phạm vi quyền hạn như thế nào?
2. Kế toán viên trong cơ quan hành chính nhà nước phải đáp ứng điều kiện về trình độ như thế nào?
3. Công việc của Kế toán viên trong cơ quan hành chính nhà nước là gì?"

### Human: Cho tôi hỏi: Mẫu quyết định cưỡng chế trả vật trong thi hành án dân sự mới nhất theo Thông tư 04/2023 / TT-BTP là mẫu nào?
### Assistant:"
1. Mẫu quyết định cưỡng chế trả vật trong thi hành án dân sự mới nhất theo Thông tư 04/2023 / TT-BTP là mẫu nào?

The results from the base model tend to be of poor quality and always re-generate the input

## Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [28]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=14428, # the number of training steps the model will take
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    # do_eval=do_eval,
    # evaluation_strategy=evaluation_strategy,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    eval_dataset=val_dataset,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


## Let's start the training process

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss
100,1.449100
200,0.811500
300,0.768500
400,0.747800
500,0.768200
600,0.760100
700,0.723300
800,0.730500
900,0.739100
1000,0.696600


## Merge and Share

After fine-tuning, if you want to merge the model with LoRA weights or share it with the Hugging Face Model Hub, you can do so. This step is optional and depends on your specific use case.

In [44]:
# Empty VRAM
import gc
del base_model
gc.collect()

del trainer
gc.collect()

0

In [45]:
torch.cuda.empty_cache() # PyTorch thing

In [46]:
gc.collect()

0

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)